In [40]:
!pip install opencv-python
!pip install twilio

1. Imports and Twilio Configuration

In [42]:
import numpy as np
import cv2
import time
import datetime
from collections import deque
from twilio.rest import Client
import os

# Twilio credentials (replace with your actual Twilio details)
account_sid = ''  # Replace with your Account SID
auth_token = ''    # Replace with your Auth Token
your_num = ''    # Replace with your phone number
twilio_num = ''   # Replace with your Twilio trial number

2. Twilio Message Sending Function

In [44]:
# Function to send a message using Twilio
def send_message(body):
    client = Client(account_sid, auth_token)
    message = client.messages.create(
        to=your_num,
        from_=twilio_num,
        body=body
    )

3. Person Detection Function

In [46]:
# Function to check if a person is detected in the frame
def is_person_present(frame, thresh=1100):
    global foog

    # Apply background subtraction
    fgmask = foog.apply(frame)

    # Get rid of shadows
    ret, fgmask = cv2.threshold(fgmask, 250, 255, cv2.THRESH_BINARY)

    # Morphological operations for noise reduction
    fgmask = cv2.dilate(fgmask, kernel, iterations=4)

    # Detect contours
    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Check for the largest contour and ensure it's larger than the threshold
    if contours and cv2.contourArea(max(contours, key=cv2.contourArea)) > thresh:
        cnt = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(cnt)
        # Modernized detection box color and font
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 150, 255), 3)  # Sleek orange box
        cv2.putText(frame, 'Person Detected', (x, y - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)  # Improved font
        return True, frame
    else:
        return False, frame

4. Setup and Initialization

In [48]:
# Set up video capture, video writer, and background subtractor
cap = cv2.VideoCapture('http://192.168.0.102:8080/video')  # Replace with your IP camera URL
foog = cv2.createBackgroundSubtractorMOG2(detectShadows=True, varThreshold=100, history=2000)

# Ensure the outputs directory exists
output_dir = 'C:/Users/Adity/OneDrive/Desktop/outputs'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Window settings for resizing
cv2.namedWindow('Live Video Feed', cv2.WINDOW_NORMAL)

# Get width and height of the frame
width = int(cap.get(3))
height = int(cap.get(4))

# Video status flags
status = False
patience = 7
detection_thresh = 15
initial_time = None
de = deque([False] * detection_thresh, maxlen=detection_thresh)

# FPS and timing
fps = 0
frame_counter = 0
start_time = time.time()

# Kernel for morphological operations
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))  # Example kernel

5. Main Loop

In [50]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Check if a person is detected in the frame
    detected, annotated_image = is_person_present(frame)

    # Register the detection status in deque
    de.appendleft(detected)

    # Logic for detecting a person
    if sum(de) == detection_thresh and not status:
        status = True
        entry_time = datetime.datetime.now().strftime("%A, %I-%M-%S %p %d %B %Y")
        try:
            out = cv2.VideoWriter(f'{output_dir}/{entry_time}.mp4', cv2.VideoWriter_fourcc(*'XVID'), 15.0, (width, height))
        except Exception as e:
            print(f"Error initializing video writer: {e}")
            continue

    # Handle logic when person leaves the frame
    if status and not detected:
        if sum(de) > (detection_thresh / 2):
            if initial_time is None:
                initial_time = time.time()
        elif initial_time is not None:
            if time.time() - initial_time >= patience:
                status = False
                exit_time = datetime.datetime.now().strftime("%A, %I:%M:%S %p %d %B %Y")
                out.release()
                initial_time = None
                body = f"Alert: \nA Person Entered the Room at {entry_time}\nLeft the room at {exit_time}"
                send_message(body)

    # Reset initial_time if detection count goes up
    elif status and sum(de) > (detection_thresh / 2):
        initial_time = None

    # Display current time, FPS, day, and room status
    current_time = datetime.datetime.now().strftime("%I:%M:%S %p")
    current_day = datetime.datetime.now().strftime("%A")
    date = datetime.datetime.now().strftime("%d %B, %Y")
    cv2.putText(annotated_image, f'Time: {current_time}', (10, 50), cv2.FONT_HERSHEY_PLAIN, 1.0, (255, 255, 255), 2)
    cv2.putText(annotated_image, f'Day: {current_day}', (10, 90), cv2.FONT_HERSHEY_PLAIN, 1.0, (255, 255, 255), 2)
    cv2.putText(annotated_image, f'Date: {date}', (10, 130), cv2.FONT_HERSHEY_PLAIN, 1.0, (255, 255, 255), 2)
    cv2.putText(annotated_image, f'Room Occupied: {status}', (10, 20), cv2.FONT_HERSHEY_PLAIN, 1.1, (0, 255, 0), 2)

    # Display FPS and detection status
    fps_text = f'FPS: {fps:.2f}'
    cv2.putText(annotated_image, fps_text, (width - 200, height - 50), cv2.FONT_HERSHEY_PLAIN, 1.0, (255, 255, 255), 2)

    # Display patience countdown if needed
    if initial_time is None:
        patience_text = f'Patience: {patience}'
    else:
        patience_text = f'Patience: {max(0, patience - (time.time() - initial_time)):.2f}'
    cv2.putText(annotated_image, patience_text, (width - 250, height - 20), cv2.FONT_HERSHEY_PLAIN, 1.0, (0, 255, 255), 2)

    # Save frame if a person is detected
    if status:
        out.write(annotated_image)

    # Show the frame with updated window title
    cv2.imshow('Live Video Feed', annotated_image)

    # Calculate FPS
    frame_counter += 1
    fps = frame_counter / (time.time() - start_time)

    # Exit on 'q' key press
    if cv2.waitKey(30) == ord('q'):
        break

6. Cleanup

In [52]:
# Cleanup
cap.release()
cv2.destroyAllWindows()
if status:
    out.release()

END